In [127]:
!pip install transformers datasets evaluate


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [128]:
from huggingface_hub import notebook_login

notebook_login()

In [129]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="https://raw.githubusercontent.com/lwalton101/ProjectNigel/transformers/Brain/intent/train/train_data.csv")

In [130]:
unique_sentiments = dataset['train'].unique('label')
unique_sentiments

['Greeting', 'PlayMusic']

In [131]:
label2id = {sentiment: i for i, sentiment in enumerate(unique_sentiments)}
print(label2id)
id2label = {sentiment: i for i, sentiment in label2id.items()}
print(id2label)

{'Greeting': 0, 'PlayMusic': 1}
{0: 'Greeting', 1: 'PlayMusic'}


In [132]:
def map_sentiment(example):
    return {'label': label2id[example['label']]}
dataset['train'] = dataset['train'].map(map_sentiment)

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

In [133]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [134]:
def preprocess_func(examples):
    return tokenizer(examples["text"], truncation=True)

In [135]:
tokenized_dataset = dataset.map(preprocess_func, batched=True)

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

In [136]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [137]:
import evaluate

accuracy = evaluate.load("accuracy")

In [138]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In [139]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_dataset["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
print(total_train_steps)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

40


In [140]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [141]:
train_test_split = tokenized_dataset['train'].train_test_split(test_size=0.1)
print(train_test_split)

train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

train_valid_split = train_dataset.train_test_split(test_size=0.2)

train_dataset = train_valid_split['train']
valid_dataset = train_valid_split['test']
print(valid_dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 116
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 13
    })
})
Dataset({
    features: ['label', 'text', 'input_ids', 'attention_mask'],
    num_rows: 24
})


In [147]:
tf_train_set = model.prepare_tf_dataset(
    train_dataset,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    valid_dataset,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [148]:
model.compile(optimizer=optimizer)

In [153]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [154]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="IntentDetection",
    tokenizer=tokenizer,
)

c:\Users\lukew\Documents\Products\Python\ProjectNigel\Brain\.venv\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Nicknotname/IntentDetection into local empty directory.


In [ ]:
callbacks = [metric_callback, push_to_hub_callback]
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)